In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nslkdd/KDDTest+.arff
/kaggle/input/nslkdd/KDDTest-21.arff
/kaggle/input/nslkdd/KDDTest1.jpg
/kaggle/input/nslkdd/KDDTrain+.txt
/kaggle/input/nslkdd/KDDTrain+_20Percent.txt
/kaggle/input/nslkdd/KDDTest-21.txt
/kaggle/input/nslkdd/KDDTest+.txt
/kaggle/input/nslkdd/KDDTrain+.arff
/kaggle/input/nslkdd/index.html
/kaggle/input/nslkdd/KDDTrain+_20Percent.arff
/kaggle/input/nslkdd/KDDTrain1.jpg
/kaggle/input/nslkdd/nsl-kdd/KDDTest+.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTest-21.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTest1.jpg
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+_20Percent.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTest-21.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTest+.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+.arff
/kaggle/input/nslkdd/nsl-kdd/index.html
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+_20Percent.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTrain1.jpg


In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBClassifier, XGBRegressor
from scipy.io import arff
from sklearn.preprocessing import StandardScaler

In [3]:
dataset = pd.read_csv('/kaggle/input/nslkdd/KDDTrain+.txt')
dataset.columns = ['duration','protocol_type','service','flag',
'src_bytes','dst_bytes','land','wrong_fragment','urgent','hot'
,'num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root',
'num_file_creations','num_shells','num_access_files','num_outbound_cmds',
'is_host_login','is_guest_login','count','srv_count','serror_rate','srv_serror_rate',
'rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate',
'dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate',
'dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate'
,'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate',
'attack','level']

In [4]:
dataset

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack,level
0,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
1,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
2,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
3,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
4,0,tcp,private,REJ,0,0,0,0,0,0,...,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20
125968,8,udp,private,SF,105,145,0,0,0,0,...,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal,21
125969,0,tcp,smtp,SF,2231,384,0,0,0,0,...,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal,18
125970,0,tcp,klogin,S0,0,0,0,0,0,0,...,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20


In [4]:
dataset = pd.read_csv('/kaggle/input/nslkdd/KDDTrain+.txt')
testset = pd.read_csv('/kaggle/input/nslkdd/KDDTest+.txt')
dataset.columns = ['duration','protocol_type','service','flag',
'src_bytes','dst_bytes','land','wrong_fragment','urgent','hot'
,'num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root',
'num_file_creations','num_shells','num_access_files','num_outbound_cmds',
'is_host_login','is_guest_login','count','srv_count','serror_rate','srv_serror_rate',
'rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate',
'dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate',
'dst_host_same_src_port_rate','dst_host_srv_diff_host_rate','dst_host_serror_rate'
,'dst_host_srv_serror_rate','dst_host_rerror_rate','dst_host_srv_rerror_rate',
'attack','level']
testset.columns = dataset.columns
newset = pd.concat([dataset, testset])

40

In [7]:
Xset = newset.drop('attack', axis=1)
Yset = newset.attack

In [5]:
forTest = Xset.head(30)
forTest

NameError: name 'Xset' is not defined

In [9]:
def find_top_n_rows_with_most_distinct_values(df, n=5):
    # Apply a lambda function to each row to count the distinct values
    distinct_counts = df.apply(lambda row: len(set(row)), axis=1)
    
    # Sort the rows by the number of distinct values in descending order
    top_n_indices = distinct_counts.nlargest(n).index
    
    # Return the top N rows with the most distinct values and their distinct counts
    return df.loc[top_n_indices], distinct_counts.loc[top_n_indices]

# Example usage
top_5_rows, top_5_distinct_counts = find_top_n_rows_with_most_distinct_values(Xset, n=5)

idd = list(top_5_distinct_counts)
print("\nDistinct value counts for the top 5 rows:\n", idd)


Distinct value counts for the top 5 rows:
 [10, 25, 12, 24, 23, 12, 23, 14, 23]


In [10]:
Xset.iloc[22344: 22345, :]

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,level
22344,0,tcp,http,SF,180,255639,0,0,0,0,...,255,1.0,0.0,0.01,0.0,0.0,0.0,0.0,0.0,21


In [12]:
Xset.columns

Index(['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'level'],
      dtype='object')

In [17]:
126//42

3

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from sklearn.preprocessing import LabelEncoder
import shutil

# Automatically map categorical values to colors
def generate_categorical_colors(df, categorical_columns):
    color_map = {}

    for col in categorical_columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        num_categories = len(df[col].unique())
        cmap = plt.cm.get_cmap('hsv', num_categories)
        color_map[col] = {category: tuple(map(int, np.array(cmap(idx)[:3]) * 255))
                          for idx, category in enumerate(range(num_categories))}

    return color_map

def value_to_color(value, col_type, color_map=None, col_name=None):
    if col_type == 'numeric':
        # Map numerical values to a color (using a colormap, e.g., 'viridis')
        cmap = plt.cm.viridis
        normalized_value = (value - min_val) / (max_val - min_val)
        return tuple(map(int, np.array(cmap(normalized_value)[:3]) * 255))
    elif col_type == 'categorical' and color_map:
        # Map categorical values to their corresponding colors
        return color_map[col_name].get(value, (0, 0, 0))  # Default to black

def row_to_image(row, col_types, color_map, img_size=(12600, 1000)):
    img = np.zeros((img_size[1], img_size[0], 3), dtype=np.uint8)
    col_width = img_size[0] // len(row)

    for i, col in enumerate(row.index):
        value = row[col]
        color = value_to_color(value, col_types[col], color_map, col)

        # Fill in the column of the image with the color
        img[:, i*col_width:(i+1)*col_width] = color

    return Image.fromarray(img)

progression = float()
def dataframe_to_images(df, col_types, color_map, output_dir='images_subclasses', img_size=(12600, 1000)):
    if os.path.exists(output_dir):
      shutil.rmtree(output_dir)
#     classes = Yset.unique()
    os.mkdir(output_dir)
#     for cls in classes:
#       os.mkdir(output_dir + '/' + cls)

    for idx, row in df.iterrows():
        img = row_to_image(row, col_types, color_map, img_size)
        attack_class = Yset.iloc[idx]
        img.save(output_dir + '/' + f'{idx}.png')
        if idx%1000 == 0:
          print(f'index : {idx} done')

# Example usage
df = Xset.iloc[0:5, :].copy() ## Should be changed

# Define column types
col_types = dict()

cat_cols = df.select_dtypes('object')
for col in df.columns:
  if col in cat_cols:
    col_types[col] = 'categorical'
  else:
    col_types[col] = 'numeric'

# Get min and max values for numerical columns
min_val, max_val = -1, +1

# Generate color mapping for categorical columns
categorical_columns = [col for col, col_type in col_types.items() if col_type == 'categorical']
color_map = generate_categorical_colors(df, categorical_columns)

# Convert each row to an image
dataframe_to_images(df, col_types, color_map)


/tmp/ipykernel_36/2704810330.py:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('hsv', num_categories)
/tmp/ipykernel_36/2704810330.py:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('hsv', num_categories)
/tmp/ipykernel_36/2704810330.py:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('hsv', num_categories)


index : 0 done


In [35]:
cat_cols = newset.select_dtypes(include = 'object')
cat_cols.columns

Index(['protocol_type', 'service', 'flag', 'attack'], dtype='object')

In [37]:
newset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 148515 entries, 0 to 22542
Data columns (total 43 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     148515 non-null  int64  
 1   protocol_type                148515 non-null  object 
 2   service                      148515 non-null  object 
 3   flag                         148515 non-null  object 
 4   src_bytes                    148515 non-null  int64  
 5   dst_bytes                    148515 non-null  int64  
 6   land                         148515 non-null  int64  
 7   wrong_fragment               148515 non-null  int64  
 8   urgent                       148515 non-null  int64  
 9   hot                          148515 non-null  int64  
 10  num_failed_logins            148515 non-null  int64  
 11  logged_in                    148515 non-null  int64  
 12  num_compromised              148515 non-null  int64  
 13  root_

In [8]:
newset.iloc[:20, :].to_csv('newset 1st 20', index = False)